# Primer - BBC

* Uporabimo bazo [`bbc.db`](bbc.db).
* Izpišimo imena vseh evropskih držav.

Uvozimo podatkovni vmesnik in se povežimo na bazo.

In [30]:
import sqlite3 as dbapi
povezava = dbapi.connect("bbc.db")

Zapišimo naš ukaz in ga izvedimo.

In [31]:
kazalec = povezava.cursor()
sql = "SELECT * FROM bbc WHERE region = 'Europe'"
kazalec.execute(sql)        # izvedemo ukaz

Preberimo celoten rezultat v seznam in izpišimo imena držav.

In [8]:
zapisi = kazalec.fetchall() # preberemo tabelo z rezultati
for vrstica in zapisi:
    print(vrstica[0])       # ime je prvi element nabora

In [9]:
zapisi

[]

Namesto tega bi lahko z zanko `for` brali zapise enega po enega.

In [21]:
for ime, regija, povrsina, prebivalstvo, bdp in kazalec:
    print(ime, povrsina)

Croatia 56594
Cyprus 9250
Czech Republic 78866
Denmark 43098
Estonia 45227
Finland 338145
Former Yugoslav Republic of Macedonia 25713
France 543965
Georgia 69700
Germany 357027
Greece 131957
Hungary 93030
Iceland 103000
Ireland 70182
Italy 301338
Latvia 64589
Liechtenstein 160
Lithuania 65300
Luxembourg 2586
Malta 316
Moldova 33800
Monaco 2
Norway 323759
Poland 312685
Portugal 92345
Romania 238391
Russia 17000000
San Marino 61
Serbia and Montenegro 102173
Slovakia 49033
Slovenia 20273
Spain 505988
Sweden 449964
Switzerland 41284
The Netherlands 41864
Turkey 779452
Ukraine 603700
United Kingdom 242514
Vatican 0


Lahko pa tudi ročno beremo vrstico po vrstico.

In [33]:
kazalec.fetchone()

ProgrammingError: Cannot operate on a closed database.

In [23]:
next(kazalec)

StopIteration: 

Pristope lahko tudi kombiniramo.

In [26]:
for ime, *_, bdp in kazalec:
    print(ime, bdp)
    if ime == 'Slovenia':
        break
print(kazalec.fetchmany(5))
print(kazalec.fetchall())

Albania 6656000000
Andorra None
Armenia 3360000000
Austria 261630000000
Azerbaijan None
Belarus 20776000000
Belgium 319609000000
Bosnia-Hercegovina 8568000000
Bulgaria 21372000000
Croatia 28996000000
Cyprus 14187060000
Czech Republic 93330000000
Denmark 219510000000
Estonia 9113000000
Finland 170508000000
Former Yugoslav Republic of Macedonia 4700000000
France 1826463000000
Georgia 5200000000
Germany 2484900000000
Greece 182710000000
Hungary 81046000000
Iceland 11354280000
Ireland 137120000000
Italy 1494064000000
Latvia None
Liechtenstein None
Lithuania 19516000000
Luxembourg 26146950000
Malta 4863250000
Moldova 3053000000
Monaco None
Norway 239338000000
Poland 234465000000
Portugal 150675000000
Romania 64824000000
Russia 482515000000
San Marino None
Serbia and Montenegro 27510000000
Slovakia 34992000000
Slovenia 29620000000
[('Spain', 'Europe', 505988, 44100000, 935361000000), ('Sweden', 'Europe', 449964, 8900000, 318353000000), ('Switzerland', 'Europe', 41284, 7100000, 342433000000),

Nazadnje zaprimo kazalec in povezavo na bazo.

In [32]:
#kazalec.close()
povezava.close()

# Še en primer

In [35]:
import sqlite3 as dbapi
# Povežemo se na novo bazo - s tem jo ustvarimo
conn = dbapi.connect("testdb.sqlite")
cur = conn.cursor()                        # Odpremo kazalec
cur.execute("DROP TABLE IF EXISTS test;")  # Zbrišemo tabelo, če že obstaja
# Izvedemo ukaz - ustvarimo tabelo
cur.execute("""
      CREATE TABLE test (
        id   integer PRIMARY KEY AUTOINCREMENT,
        num  integer,
        data text
      );
    """)
# Vstavimo podatke v tabelo
cur.execute("INSERT INTO test (num, data) VALUES (100, 'KU-KU');")
cur.execute("SELECT * FROM test;")         # Preberemo zapisane podatke
rezultat  = cur.fetchone()                 # Hočemo le eno vrstico
print(rezultat)                            # Izpiše se (1, 100, "KU-KU")
conn.commit()                              # Poskrbimo, da so spremembe trajne
# Zapremo povezave z bazo
cur.close()
conn.close()

(1, 100, 'KU-KU')


Kaj pa, če gre kaj narobe? V vsakem primeru želimo zapreti povezave - to lahko dosežemo z blokom `finally`.

In [36]:
import sqlite3 as dbapi
# Povežemo se na novo bazo - s tem jo ustvarimo
conn = dbapi.connect("testdb.sqlite")
try:
    cur = conn.cursor()                            # Odpremo kazalec
    try:
        cur.execute("DROP TABLE IF EXISTS test;")  # Zbrišemo tabelo, če že obstaja
        # Izvedemo ukaz - ustvarimo tabelo
        cur.execute("""
              CREATE TABLE test (
                id   integer PRIMARY KEY AUTOINCREMENT,
                num  integer,
                data text
              );
            """)
        # Vstavimo podatke v tabelo
        cur.execute("INSERT INTO test (num, data) VALUES (100, 'KU-KU');")
        cur.execute("SELECT * FROM test;")         # Preberemo zapisane podatke
        rezultat  = cur.fetchone()                 # Hočemo le eno vrstico
        print(rezultat)                            # Izpiše se (1, 100, "KU-KU")
        # Vstavimo še eno vrstico
        cur.execute("INSERT INTO test (id, num, data) VALUES (1, 200, 'kaj pa zdaj?');")
        # Tukaj pride do napake, saj bi se ponovila vrednost v stolpcu id
        conn.commit()                              # Poskrbimo, da so spremembe trajne
    finally:
        # Zapremo povezavo s kazalcem
        print("Zapiramo kazalec")
        cur.close()
except dbapi.IntegrityError as ex:
    conn.rollback()                                # Prekličemo spremembe
    print(f"Prišlo je do napake: {ex}!")
    cur = conn.cursor()                            # Odpremo nov kazalec, ker smo starega zaprli
    cur.execute("SELECT * FROM test;")             # Preberemo zapisane podatke
    rezultat  = cur.fetchone()                     # Hočemo le eno vrstico
    print(rezultat)                                # Ni take vrstice, dobimo None
    cur.close()
finally:
    # Zapremo povezavo z bazo
    print("Zapiramo povezavo z bazo")
    conn.close()

(1, 100, 'KU-KU')
Zapiramo kazalec
Prišlo je do napake: UNIQUE constraint failed: test.id!
None
Zapiramo povezavo z bazo


# Uporaba `with`

Z `ẁith` poskrbimo za potrditev transakcije ob uspešnem izvajanju oziroma za preklic ob napaki.

In [9]:
import sqlite3 as dbapi
conn = dbapi.connect("testdb.sqlite")
cur = conn.cursor()
try:
    #cur.execute("INSERT INTO test (id, num, data) VALUES (7, 400, 'nekaj drugega');")
    with conn:
        cur.execute("INSERT INTO test (num, data) VALUES (100, 'prva vrstica');")
        cur.execute("""
            INSERT INTO test (id, num, data)
            VALUES (2, 200, 'kaj pa zdaj?');
            """)
        cur.execute("INSERT INTO test (num, data) VALUES (300, 'še nekaj');")
except dbapi.IntegrityError as ex:
    print(f"Napaka: {ex}")

Napaka: UNIQUE constraint failed: test.id


Poglejmo, kaj se nahaja v tabeli.

In [10]:
cur.execute("SELECT * FROM test;")
cur.fetchall()

[(1, 100, 'prva vrstica'),
 (2, 200, 'kaj pa zdaj?'),
 (3, 300, 'še nekaj'),
 (4, 400, 'nekaj drugega'),
 (5, 100, 'prva vrstica'),
 (6, 300, 'še nekaj')]

Kazalec in povezava sta ostala odprta, tako da ju na koncu zapremo.

In [8]:
cur.close()
conn.close()

# Parametriziranje ukazov SQL

Parametre lahko vstavljamo v ukaze SQL tako, da na njihova mesta zapišemo `?` ali ime parametra za `:`, njihove vrednosti pa podamo z drugim parametrom metode `execute`.

In [1]:
import sqlite3 as dbapi
conn = dbapi.connect("bbc.db")

In [7]:
def drzava(conn, ime):
    cur = conn.cursor()
    try:
        cur.execute("SELECT * FROM bbc WHERE name = ?;", (ime, ))
        return cur.fetchone()
    finally:
        cur.close()

In [9]:
drzava(conn, 'Germany')

('Germany', 'Europe', 357027, 82500000, 2484900000000)

In [10]:
def prebivalstvo(conn, min, max):
    cur = conn.cursor()
    try:
        cur.execute("""
              SELECT * FROM bbc
               WHERE population BETWEEN ? AND ?;
            """, [min, max])
        return cur.fetchall()
    finally:
        cur.close()

In [11]:
prebivalstvo(conn, 1000000, 10000000)

[('Albania', 'Europe', 28728, 3200000, 6656000000),
 ('Armenia', 'Europe', 29743, 3000000, 3360000000),
 ('Austria', 'Europe', 83871, 8100000, 261630000000),
 ('Azerbaijan', 'Europe', 86600, 8500000, None),
 ('Belarus', 'Europe', 207595, 9800000, 20776000000),
 ('Benin', 'Africa', 112622, 7100000, 3763000000),
 ('Bhutan', 'South Asia', 38364, 2400000, 1824000000),
 ('Bolivia', 'South America', 1100000, 9100000, None),
 ('Bosnia-Hercegovina', 'Europe', 51129, 4200000, 8568000000),
 ('Botswana', 'Africa', 581730, 1800000, 7812000000),
 ('Bulgaria', 'Europe', 110994, 7800000, 21372000000),
 ('Burundi', 'Africa', 27816, 7300000, None),
 ('Central African Republic', 'Africa', 622984, 3900000, None),
 ('Chad', 'Africa', 1280000, 9100000, 2366000000),
 ('Costa Rica', 'Americas', 51100, 4300000, None),
 ('Croatia', 'Europe', 56594, 4400000, 28996000000),
 ('Denmark', 'Europe', 43098, 5400000, 219510000000),
 ('Dominican Republic', 'Americas', 48072, 9000000, None),
 ('El Salvador', 'Americas',

In [13]:
def prebivalstvo_povrsina(conn, vrednost):
    cur = conn.cursor()
    try:
        cur.execute("""
              SELECT * FROM bbc
               WHERE population >= :vrednost AND
                     area >= :vrednost;
            """, {'vrednost': vrednost})
        return cur.fetchall()
    finally:
        cur.close()

In [14]:
prebivalstvo_povrsina(conn, 2000000)

[('Algeria', 'Middle East', 2400000, 32900000, 75012000000),
 ('Argentina', 'South America', 2800000, 39300000, 146196000000),
 ('Australia', 'Asia-Pacific', 7700000, 20300000, 546070000000),
 ('Brazil', 'South America', 8550000, 182800000, 564852000000),
 ('Canada', 'North America', 9900000, 32000000, 908480000000),
 ('China', 'Asia-Pacific', 9600000, 1300000000, 1677000000000),
 ('Democratic Republic of Congo', 'Africa', 2340000, 56000000, 6720000000),
 ('India', 'South Asia', 3100000, 1100000000, 682000000000),
 ('Kazakhstan', 'Asia-Pacific', 2700000, 15400000, None),
 ('Russia', 'Europe', 17000000, 141500000, 482515000000),
 ('Saudi Arabia', 'Middle East', 2240000, 25600000, 267008000000),
 ('Sudan', 'Middle East', 2500000, 35000000, 18550000000),
 ('United States of America',
  'North America',
  9800000,
  295000000,
  12213000000000)]

In [15]:
conn.close()

# SQL injection

Če nismo pazljivi pri vstavljanju podatkov v stavke SQL, lahko zlonameren uporabnik doseže, da se izvedejo ukazi, ki jih razvijalec aplikacije ni predvidel.

Denimo, da imamo v bazi tabelo uporabnikov, ki hrani njihova uporabniška imena, gesla in zastavico, ali gre za administratorja.

**Opomba:** v praksi gesel nikoli ne hranimo v bazi v čisti obliki!

In [48]:
import sqlite3 as dbapi
conn = dbapi.connect("testdb.sqlite")
conn.set_trace_callback(print)
cur = conn.cursor()

try:
    with conn:
        cur.executescript("""
              DROP TABLE IF EXISTS uporabnik;
              CREATE TABLE uporabnik (
                id              integer PRIMARY KEY AUTOINCREMENT,
                uporabnisko_ime text    NOT NULL UNIQUE,
                geslo           text    NOT NULL,
                admin           integer NOT NULL DEFAULT 0
              );
            """)
        cur.executemany("""
              INSERT INTO uporabnik (uporabnisko_ime, geslo, admin)
              VALUES (?, ?, ?)
            """, [('admin', '$kr1wn0G35l0', 1),
                  ('janez', 'geslo123', 0)])
finally:
    cur.close()


              DROP TABLE IF EXISTS uporabnik;

              CREATE TABLE uporabnik (
                id              integer PRIMARY KEY AUTOINCREMENT,
                uporabnisko_ime text    NOT NULL UNIQUE,
                geslo           text    NOT NULL,
                admin           integer NOT NULL DEFAULT 0
              );
BEGIN 

              INSERT INTO uporabnik (uporabnisko_ime, geslo, admin)
              VALUES ('admin', '$kr1wn0G35l0', 1)
            

              INSERT INTO uporabnik (uporabnisko_ime, geslo, admin)
              VALUES ('janez', 'geslo123', 0)
            
COMMIT


Denimo, da s sledečo funkcijo preverjamo podatke, ki jih uporabnik vnese v prijavni obrazec. Ob uspešni prijavi funkcija vrne par `(id, admin)`, kjer je `id` zaporedna številka uporabnika, `admin` pa zastavica, ki pove, ali je uporabnik administrator. Ob neuspešni prijavi (napačno uporabniško ime ali geslo) funkcija vrne `None`.

In [17]:
def prijava_slaba(conn, uporabnisko_ime, geslo):
    cur = conn.cursor()
    try:
        cur.execute(f"""
              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = '{uporabnisko_ime}' AND
                     geslo = '{geslo}';
            """)
        return cur.fetchone()
    finally:
        cur.close()

Preverimo delovanje funkcije na nekaj primerih.

In [18]:
prijava_slaba(conn, 'janez', 'geslo123')


              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = 'janez' AND
                     geslo = 'geslo123';


(2, 0)

In [19]:
prijava_slaba(conn, 'janez', 'napacno_geslo')


              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = 'janez' AND
                     geslo = 'napacno_geslo';


In [20]:
prijava_slaba(conn, 'micka', 'geslo123')


              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = 'micka' AND
                     geslo = 'geslo123';


Ob uporabi zgornje funkcije lahko zlonameren uporabnik sestavi take vhodne podatke, da bo funkcija vrnila podatke uporabnika `admin` z administratorskimi pravicami, čeprav zanj ne pozna gesla.

In [21]:
prijava_slaba(conn, "admin' OR 0 AND --", "")


              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = 'admin' OR 0 AND --' AND
                     geslo = '';


(1, 1)

Tudi, če uporabnik ni zlonameren, lahko morda povzroči napako v delovanju programa.

In [22]:
prijava_slaba(conn, "franci", "ges'lce")

OperationalError: near "lce": syntax error

Zapišimo popravljeno funkcijo za prijavo.

In [23]:
def prijava_dobra(conn, uporabnisko_ime, geslo):
    cur = conn.cursor()
    try:
        cur.execute("""
              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = ? AND
                     geslo = ?;
            """, (uporabnisko_ime, geslo))
        return cur.fetchone()
    finally:
        cur.close()

In [24]:
prijava_dobra(conn, 'janez', 'geslo123')


              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = 'janez' AND
                     geslo = 'geslo123';


(2, 0)

In [25]:
prijava_dobra(conn, 'janez', 'napacno_geslo')


              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = 'janez' AND
                     geslo = 'napacno_geslo';


In [26]:
prijava_dobra(conn, 'micka', 'geslo123')


              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = 'micka' AND
                     geslo = 'geslo123';


In [27]:
prijava_dobra(conn, "admin' OR 0 AND --", "")


              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = 'admin'' OR 0 AND --' AND
                     geslo = '';


In [31]:
prijava_dobra(conn, "franci", "ges'lce")


              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = 'franci' AND
                     geslo = 'ges''lce';


(3, 0)

In [51]:
conn.execute("""INSERT INTO uporabnik (uporabnisko_ime, geslo, admin)
              VALUES (?, ?, ?)""", ('micka', "ges'lce", 0)).lastrowid

INSERT INTO uporabnik (uporabnisko_ime, geslo, admin)
              VALUES ('micka', 'ges''lce', 0)


4

In [52]:
conn.execute("SELECT * FROM uporabnik").fetchall()

SELECT * FROM uporabnik


[(1, 'admin', '$kr1wn0G35l0', 1),
 (2, 'janez', 'geslo123', 0),
 (3, 'franci', "ges'lce", 0),
 (4, 'micka', "ges'lce", 0)]

In [47]:
conn.close()